In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
# 의존성 패키지 설치하기
!pip install ratsnlp

### 트랜스포머 기반 모델 입력 데이터를 모델에 제공하기 위해 필요한 주요 구성 요소

구성 요소들은 모델이 텍스트 데이터를 이해하고 처리할 수 있도록 정보를 인코딩합니다.

1. input_ids
텍스트를 토크나이즈한 후, 각 토큰을 해당하는 어휘 사전(vocabulary)의 인덱스로 변환한 것입니다. 즉, 입력 텍스트가 모델이 이해할 수 있는 숫자의 시퀀스로 변환된 형태입니다. 이러한 변환을 통해, 모델은 텍스트 데이터를 처리할 수 있게 됩니다.
2. token_type_ids
주로 BERT와 같이 입력이 여러 개의 세그먼트로 구성될 수 있는 모델에서 사용됩니다. 예를 들어, 질문-응답 태스크나 텍스트 쌍을 처리하는 태스크에서, 두 개의 다른 텍스트 세그먼트를 구분하기 위해 사용됩니다. 각 토큰이 속한 세그먼트를 나타내는 인덱스로, 대개 0과 1의 값을 가집니다. 이를 통해 모델은 두 개의 세그먼트를 구분하고, 관계를 올바르게 이해할 수 있습니다.
  - 질문-응답 태스크란 질문(question)에 답(answer)을 하는 과제입니다. 예를들어 질문에 대한 답을 지문(context)에서 찾는 걸 가리킵니다. 질의응답(Question Answering, QA) 태스크에서는 지문(context)과 질문(question)을 구분하기 위해 token_type_ids를 사용

    {
    "context": "BERT는 구글이 개발한 사전 훈련된 딥 러닝 자연어 처리 모델입니다. 이 모델은 다양한 자연어 처리 태스크에서 뛰어난 성능을 보여줍니다.",<br>
    "question": "BERT를 개발한 회사는 어디인가요?",<br>
    "answer": {
      "text": "구글",
      "start_index": 12,
      "end_index": 14
    }
  }

    [ token_type_ids ]
    
    0 (CLS에 해당), 0 (지문 내용에 해당), 0 ([SEP]에 해당), 1 (질문 내용에 해당), 1 ([SEP]에 해당)

  - 텍스트 쌍을 처리하는 태스크의 대표 예시로 자연어 추론(Natural Langugage Inference; NLI)이 있습니다. 두 개 문장이 참(entailment), 거짓(contradiction), 중립(neutral)인지 가려내는 것입니다.
3. attention_mask
모델이 특정 토큰에 주의(attention)를 기울여야 하는지를 나타내는 이진 마스크입니다. 주로 패딩(padding)된 토큰을 처리할 때 사용됩니다. 트랜스포머 모델은 일반적으로 고정된 길이의 입력을 요구하기 때문에, 짧은 텍스트를 입력할 때는 나머지 부분을 특정 값(예: 0)으로 채워 넣어야 합니다. 이러한 패딩된 부분을 모델이 실제 데이터로 처리하지 않도록 attention_mask가 사용됩니다. 패딩된 위치는 0으로, 실제 데이터가 있는 위치는 1로 설정하여 모델이 패딩 부분을 무시하도록 합니다.

이 세 가지 구성 요소는 모델이 입력 데이터를 효과적으로 처리하고, 다양한 NLP 태스크에서 좋은 성능을 발휘할 수 있도록 돕습니다.

### GPT 입력값 만들기 - 기 작성한 bpe 활용

GPT 입력값을 만들려면 토크나이저부터 준비해야 하며 아래 코드를 수행하면 GPT 모델이 사용하는 토크나이저를 초기화할 수 있다. 먼저 자신의 구글 드라이브 경로(/gdrive/My Drive/nlpbook/bbpe)에는 이전 실습에서 만든 바이트 기준 BPE 어휘 집합(vocab.json)과 바이그램 쌍의 병합 우선순위(merge.txt)가 있어야 합니다.

In [ ]:
# GPT 토크나이저 선언: GPT2Tokenizer'는 텍스트를 GPT-2 모델이 이해할 수 있는 형식으로 변환
# 여기에는 텍스트를 토큰(단어 또는 하위 단어)으로 분할하고 이러한 토큰을 모델이 처리에 사용하는 숫자 ID로 변환하는 작업이 포함
from transformers import GPT2Tokenizer # transformers 라이브러리에서 GPT2Tokenizer 클래스를 가져온다.
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe") # 사전 훈련된 토크나이저 로드
tokenizer_gpt.pad_token = "[PAD]" # 토크나이저는 패딩 목적으로 사용할 토큰을 알아야 하기 때문에 'pad_token'을 명시적으로 설정하는 것이 필요

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
# 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
# GPT 모델은 바이트 기준 BPE를 적용
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

GPT 모델 입력 만들기

In [ ]:
#  GPT 모델 입력
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length", # 문자의 최대 길이에 맞춰 패딩. max_length 매개변수로 지정된 최대 길이로 균일하게 시퀀스를 채우도록 지시
    max_length=12, # 문장의 토큰 기준 최대 길이. 문장이 12개 이상의 토큰으로 변환되면 잘림. 더 적은 수로 변환되면 패딩 토큰으로 채워진다.
    truncation=True, # 문장 잘림 허용 옵션. truncation=False로 설정되어 있어 토큰 수가 max_length보다 길 경우, 오류를 발생 가능
)

`batch_inputs`의 내용을 확인
- input_ids: 문장의 토큰화된 표현을 포함하며, 각 토큰은 토크나이저 어휘에서 해당하는 ID로 대체.
- attention_mask: 각 위치에 토큰이 존재하는지를 나타내는 이진 마스크입니다. 토큰이 존재하면 1, 토큰이 존재하지 않으면(즉, 패딩인 경우) 0입니다. 이는 모델이 실제 데이터와 패딩을 구별할 수 있도록 도와준다.
- GPT는 주어진 문맥을 기반으로 다음에 올 텍스트를 예측하는 구조이기 때문에, token_type_ids와 같은 추가적인 입력 정보가 필요하지 않습니다.

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
# 'input_ids'는 토큰화 결과를 가지고 각 토큰들을 인덱스(index)로 바꾼 것
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
# attention_mask는 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분해 알려주는 장치
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

### 사전 훈련된 KoGPT2 (한국어 GPT-2) Ver 2.0

KoGPT2 (한국어 GPT-2) Ver 2.0 : 한국어 자연어 처리를 위해 특별히 사전 훈련된 GPT-2 모델
- 한국어 특화: KoGPT2는 한국어 텍스트 데이터를 바탕으로 사전 훈련되었기 때문에 한국어 문법과 어휘에 대한 이해도가 높습니다. 이를 통해 한국어 자연어 이해 및 생성 태스크에서 뛰어난 성능을 보입니다.
- 다양한 응용 가능: KoGPT2는 텍스트 생성, 문장 완성, 챗봇 개발, 문서 요약, 번역 등 다양한 자연어 처리 작업에 활용될 수 있습니다.
사전 훈련된 모델: KoGPT2는 대규모 한국어 데이터셋을 사용하여 사전 훈련된 모델을 제공합니다. 사용자는 이 사전 훈련된 모델을 바탕으로 특정 태스크에 대한 미세 조정(fine-tuning)을 수행하여 사용할 수 있습니다.
- Transformer 기반: KoGPT2도 GPT-2와 마찬가지로 Transformer 아키텍처를 기반으로 합니다. Transformer는 주의 메커니즘(attention mechanism)을 사용하여 텍스트의 다양한 부분 사이의 관계를 효과적으로 학습할 수 있습니다.
- KoGPT2 모델은 transformers 라이브러리를 통해 쉽게 사용할 수 있으며, SKT에서 제공하는 skt/kogpt2-base-v2와 같은 모델 체크포인트를 활용하여 특정 태스크에 미세 조정할 수 있습니다.


https://github.com/SKT-AI/KoGPT2

### pad, eos (end-of-sequence), bos (beginning-of-sequence) 옵션

- 토크나이저에서의 사용은 데이터를 모델이 처리할 수 있는 형태로 준비하는 데 중점을 둡니다. 이는 입력 데이터의 전처리와 관련이 있습니다. 토크나이저는 텍스트를 모델이 이해할 수 있는 형태, 즉 토큰 ID의 시퀀스로 변환하는 역할을 합니다. 여기서 pad, eos, bos 토큰은 다음과 같은 목적으로 사용됩니다:
  - pad_token: 모델이 배치 처리를 할 때, 모든 입력 시퀀스가 동일한 길이를 갖도록 만듭니다. 짧은 시퀀스는 pad_token으로 채워집니다.
  - eos_token과 bos_token: 텍스트 시퀀스의 시작과 끝을 명시합니다. 이는 모델이 문장의 경계를 인식하는 데 도움을 주며, 특히 다양한 자연어 처리 작업에서 중요한 정보입니다.
- 모델에서의 사용은 모델의 동작과 출력의 특성을 제어하는 데 중요합니다. 이는 모델의 내부 동작과 직접적으로 관련이 있습니다. 모델은 토크나이징된 입력을 받아 자연어 처리 작업을 수행합니다 (예: 텍스트 생성, 번역, 분류 등). 모델에서 pad, eos, bos 토큰을 사용하는 것은 다음과 같은 목적을 가집니다:
  - pad_token_id: 주로 모델이 손실을 계산할 때 사용됩니다. 모델은 패딩된 부분을 무시하고 실제 유용한 데이터에만 집중해야 합니다.
  - eos_token_id와 bos_token_id: 텍스트 생성 작업에서, 모델이 문장을 시작하거나 종료할 시점을 결정하는 데 사용됩니다. 예를 들어, eos_token_id는 모델이 문장을 종료해야 할 때 사용되며, bos_token_id는 생성 작업의 시작점으로 사용될 수 있습니다.

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
# 텍스트를 토큰화하고 필요한 정보 얻기
encoded_input = tokenizer(text, return_tensors='pt', padding="max_length", max_length=12, truncation=True)

encoded_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# input_ids와 attention_mask 확인
input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

print("input_ids:", input_ids)
print("attention_mask:", attention_mask)

input_ids: tensor([[33245, 10114, 12748, 11357,     3,     3,     3,     3,     3,     3,
             3,     3]])
attention_mask: tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


gpt model 생성

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'

input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


### BERT 입력값 만들기 - 기 작성한 wordpiece 활용

- BERT (Bidirectional Encoder Representations from Transformers)는 다양한 자연어 처리(NLP) 작업에서 사용되는 인기 있는 모델. BertTokenizer는 특히 BERT 모델에 적합한 텍스트 토큰화 도구
- BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)가 있어야 한다. 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언한다.

In [ ]:
# BertTokenizer는 BERT 모델이 이해할 수 있는 형태로 텍스트를 토큰화하는 데 사용
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(  # 사전 훈련된 토크나이저 로드
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False, # 토크나이저가 텍스트를 소문자로 변환하지 않도록 지정
)

예시 문장 세 개를 각각 토큰화해보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [ ]:
print(tokenized_sentences)

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'], ['흠', '.', '.', '.', '포스터', '##보고', '초딩', '##영화', '##줄', '.', '.', '.', '.', '오버', '##연기', '##조차', '가볍', '##지', '않', '##구나'], ['별루', '였다', '.', '.']]


이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length", # max_length 매개변수로 지정된 최대 길이와 동일한 길이로 채워지도록 보장
    max_length=12, # 토큰화된 각 시퀀스가 ​​가져야 하는 고정 길이를 설정
    truncation=True, # 토크나이저가 max_length를 초과하는 시퀀스를 자를 수 있도록 허용
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [ ]:
# # BERT 모델 세 가지의 입력값
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

- [CLS] 토큰은 "분류"를 나타내며 모든 입력 시퀀스의 첫 번째 토큰으로 사용. BERT가 분류 작업(예: 감정 분석, 의도 감지)에 사용되는 경우 이 토큰의 표현은 분류 작업의 집계 시퀀스 표현으로 사용된다. 본질적으로 이는 전체 입력 시퀀스의 의미를 요약하는 역할
- [SEP] 토큰은 "구분 기호"를 나타내며 입력 내에서 개별 세그먼트를 구분하는 데 사용. 이는 질문 답변(모델이 질문과 맥락을 구별해야 하는 경우) 또는 문장 쌍 작업(예: 모델이 두 문장 간의 관계를 결정하는 자연어 추론)과 같은 여러 입력 시퀀스가 ​​포함된 작업에 특히 중요
- 예를 들어 문장 쌍 분류 작업에서 입력은 [CLS] 문장 1 [SEP] 문장 2 [SEP]와 같을 수 있으며 [SEP] 토큰은 첫 번째 문장의 끝을 표시하고 구분

In [ ]:
# 토큰 인덱스 시퀀스
# 문장 앞에 2, 끝에 3이 붙는 것은 각각 [CLS], [SEP] 라는 토큰에 대응하는 인덱스
batch_inputs['input_ids']

[[2, 620, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1245, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [ ]:
# BERT의 attention_mask는 GPT와 마찬가지로 일반 토큰이 자리한 곳(1)과 패딩 토큰이 자리한 곳(0)을 구분
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
# token_type_ids는 세그먼트(segment)에 해당하는 것으로 모두 0
# BERT 모델은 기본적으로 문서(혹은 문장) 2개를 입력받는데, token_type_ids로 구분.
# 첫 번째 세그먼트(문서 혹은 문장)에 해당하는 token_type_ids는 0, 두 번째 세그먼트는 1
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

### 사전 훈련된 KoBERT 모델 사용

In [ ]:
from transformers import BertTokenizer

# KoBERT 모델의 사전 훈련된 토크나이저 로드
tokenizer_kobert = BertTokenizer.from_pretrained(
    "monologg/kobert",  # Hugging Face 모델 허브에서 KoBERT 모델을 지정
    do_lower_case=False,  # 토크나이저가 텍스트를 소문자로 변환하지 않도록 지정
)

# 사용 예시
text = "한국어 모델을 사용하여 텍스트 처리를 해봅시다."
input_ids = tokenizer_kobert.encode(text, add_special_tokens=True)
print(input_ids)


vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

[2, 0, 0, 0, 0, 0, 0, 54, 3]


In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_kobert.tokenize(sentence) for sentence in sentences]
print(tokenized_sentences)

[['아', '[UNK]', '.', '.', '진짜', '[UNK]', '[UNK]'], ['흠', '.', '.', '.', '[UNK]', '[UNK]', '.', '.', '.', '.', '[UNK]', '[UNK]', '[UNK]'], ['[UNK]', '였다', '.', '.']]


In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_kobert(
    sentences,
    padding="max_length", # max_length 매개변수로 지정된 최대 길이와 동일한 길이로 채워지도록 보장
    max_length=12, # 토큰화된 각 시퀀스가 ​​가져야 하는 고정 길이를 설정
    truncation=True, # 토크나이저가 max_length를 초과하는 시퀀스를 자를 수 있도록 허용
    return_tensors='pt'
)

batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(batch_inputs.input_ids)
print(batch_inputs.token_type_ids)
print(batch_inputs.attention_mask)

tensor([[   2, 6797,    0,   54,   54, 7347,    0,    0,    3,    1,    1,    1],
        [   2, 7989,   54,   54,   54,    0,    0,   54,   54,   54,   54,    3],
        [   2,    0, 6946,   54,   54,    3,    1,    1,    1,    1,    1,    1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
